In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_bureau_feats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy
from sklearn.metrics import f1_score

In [2]:
import sys
sys.path.append('ml_lib/')
from encoding import FreqeuncyEncoding
from custom_classifier_mutliclass import Estimator
from hyperopt_multiclass import HyperOptModelSelection
from hyperopt import hp
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

In [3]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [4]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle('bureau_future_feats_corrected.pkl')
# bur_df2 = pd.read_pickle('bureau_loan_duration_hist_str_feats.pkl')
bur_df2 = pd.read_pickle('ltfs_bureau_future_feats.pkl')

In [5]:
target_map = {'No Top-up Service': 0,
 '12-18 Months': 1,
 '18-24 Months': 2,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '36-48 Months': 5,
 ' > 48 Months': 6,
 }
train['Top-up Month'] = train['Top-up Month'].map(target_map)

In [6]:
reverse_map = {v:k for k,v in target_map.items()}

In [7]:
reverse_map

{0: 'No Top-up Service',
 1: '12-18 Months',
 2: '18-24 Months',
 3: '24-30 Months',
 4: '30-36 Months',
 5: '36-48 Months',
 6: ' > 48 Months'}

In [8]:
train['Top-up Month'].value_counts()

0    106677
6      8366
5      3656
3      3492
4      3062
2      2368
1      1034
Name: Top-up Month, dtype: int64

In [9]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

In [10]:
bur_df.shape

(139964, 112)

In [11]:
df = df.merge(bur_df,on='ID',how='left').merge(bur_df2,on='ID',how='left')
# df = df.merge(bur_df,on='ID',how='left')
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'])
df['MaturityDAte'] = pd.to_datetime(df['MaturityDAte'])

In [12]:
df.shape

(143400, 256)

In [13]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_mean_correctedCURRE

In [14]:
df['pin1'] = df['ZiPCODE'].fillna('000000').astype('str').str[:2]
df['pin2'] = df['ZiPCODE'].fillna('000000').astype('str').str[2:4]
df['pin3'] = df['ZiPCODE'].fillna('000000').astype('str').str[4:6:]
df['cal_tenor'] = (df['MaturityDAte'] - df['DisbursalDate']).dt.days
df['emi_sal_ratio'] = df['EMI']/df['MonthlyIncome']
df['ltv2'] = df['DisbursalAmount']/df['AssetCost']

In [15]:
df['disb_year'] = df['DisbursalDate'].dt.year
df['disb_mon'] = df['DisbursalDate'].dt.month
df['disb_day'] = df['DisbursalDate'].dt.day
df['disb_dow'] = df['DisbursalDate'].dt.dayofweek

df['mat_year'] = df['MaturityDAte'].dt.year
df['mat_mon'] = df['MaturityDAte'].dt.month
df['mat_day'] = df['MaturityDAte'].dt.day
df['mat_dow'] = df['MaturityDAte'].dt.dayofweek

In [16]:
cat_cols = ['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'Area','ManufacturerID','SupplierID','pin1','pin2','pin3', 'SEX', 'City', 'State','BranchID']
target = 'Top-up Month'
drop_cols = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID','ZiPCODE']
num_cols = (df.columns[~df.columns.isin([target]+drop_cols+cat_cols)].tolist())
use_cols = cat_cols+num_cols

In [17]:
len(use_cols)

263

In [18]:
fe = FreqeuncyEncoding(categorical_columns=cat_cols,normalize=True,return_df=True)
df = fe.fit_transform(df)

In [19]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_mean_correctedCURRE

In [20]:
train = df[df.ID.isin(train.ID)]
test = df[df.ID.isin(test.ID)]
train = train.sort_values('ID',ascending=True)
test = test.sort_values('ID',ascending=True)
train[target] = train[target].astype('int')

In [21]:
train.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT,ltfs_mean_correctedCURRE

In [22]:
folds = StratifiedKFold(5, shuffle = True, random_state = 2)
folds = [(x,y) for x,y in folds.split(train,train[target])]

In [23]:
lgbm_space = {
    "n_estimators": 5000,
    "num_leaves": hp.quniform("num_leaves", 16, 128, 16),
    "min_child_weight": hp.quniform("min_child_weight", 10, 100, 20),
    "subsample": hp.quniform("subsample", 0.4, 1, 0.2),
    "colsample_bytree": hp.quniform("colsample_bytree", 0.4, 1, 0.2),
    "subsample_freq": 5,
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": -1,
}

In [24]:
est = Estimator(model = LGBMClassifier(),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [25]:
%%time
hyp= HyperOptModelSelection(model=est,space=lgbm_space,max_evals=50,is_maximize=True,log_file_path='hyp2.txt')

CPU times: user 166 µs, sys: 31 µs, total: 197 µs
Wall time: 149 µs


In [39]:
# %%time
# hyp.fit(train[use_cols].values,train[target].values)

In [25]:
est = hyp.best_estimator

In [28]:
est.model

LGBMClassifier(colsample_bytree=0.7000000000000001, min_child_weight=60.0,
               n_estimators=5000, num_leaves=64, objective='multiclass',
               subsample_freq=5)

In [27]:
params={'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 40.0, 'n_estimators': 5000,
        'n_jobs': -1, 'num_leaves': 96, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

In [25]:
est = Estimator(model = LGBMClassifier(colsample_bytree=0.7000000000000001, min_child_weight=60.0,
               n_estimators=5000, num_leaves=64, objective='multiclass',
               subsample_freq=5),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

# est = Estimator(model = LGBMClassifier(**params),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [26]:
temp = est.fit_transform(train[use_cols].values,train[target].values)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.357438	valid_1's multi_logloss: 0.273457
[200]	valid_0's multi_logloss: 0.358604	valid_1's multi_logloss: 0.215333
Early stopping, best iteration is:
[125]	valid_0's multi_logloss: 0.356954	valid_1's multi_logloss: 0.256419
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.354481	valid_1's multi_logloss: 0.274154
[200]	valid_0's multi_logloss: 0.354623	valid_1's multi_logloss: 0.216113
Early stopping, best iteration is:
[130]	valid_0's multi_logloss: 0.353825	valid_1's multi_logloss: 0.25424
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.351392	valid_1's multi_logloss: 0.274938
[200]	valid_0's multi_logloss: 0.351651	valid_1's multi_logloss: 0.216934
Early stopping, best iteration is:
[129]	valid_0's multi_logloss: 0.350459	valid_1's multi_logloss: 0.255546
Training until validation scores don't impro

In [27]:
est.cv_scores,est.overall_cv_score

([0.6128458238309253,
  0.6156782436289264,
  0.6271455417909506,
  0.6188124724720626,
  0.6255181062105389],
 0.6201677086954925)

In [41]:
est.feature_importance_df(use_cols).head(50)

,column,feature_importance,rank
21,MonthlyIncome,0.024682,1
252,cal_tenor,0.023604,2
99,total_sanctioned_amount_Gold Loan,0.019779,3
253,emi_sal_ratio,0.017608,4
11,City,0.016900,5
12,State,0.016709,6
6,SupplierID,0.016231,7
49,min_tenor,0.016097,8
164,ltfs_min_tenor,0.016051,9
9,pin3,0.015855,10


In [28]:
pd.Series(temp.argmax(axis=1)).value_counts(1)

0    0.901745
6    0.038856
5    0.016027
3    0.014893
4    0.012895
2    0.010835
1    0.004749
dtype: float64

In [32]:
f1_score(train[target],pd.Series(temp.argmax(axis=1)),average='macro')

0.6201677086954925

In [33]:
test_preds = [est.predict_proba(test[use_cols].values) for est in est.fitted_models]

In [34]:
test['Top-up Month'] = np.sum(test_preds,axis=0).argmax(axis=1)

In [35]:
test['Top-up Month'].value_counts(1)

0    0.922889
6    0.041234
5    0.015124
4    0.007325
3    0.005697
2    0.005019
1    0.002713
Name: Top-up Month, dtype: float64

In [36]:
sub = test[['ID','Top-up Month']]

In [37]:
sub['Top-up Month'] = sub['Top-up Month'].map(reverse_map)

In [38]:
sub.to_csv('LGBM_CV0.62.csv',index=False)

In [79]:
# est.transform(test[use_cols].values)

In [61]:
# est.feature_importance_df(use_cols)

CPU times: user 312 µs, sys: 1 µs, total: 313 µs
Wall time: 221 µs


In [ ]:
f1_score(average='macro')

Starting HyperOpt 50 Evals with Dataset of Shape ((128655, 140),(128655,))


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


Iteration: 1, Training with params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'min_child_weight': 100.0, 'n_estimators': 5000, 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 0.6000000000000001, 'subsample_freq': 5}


Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.386726	valid_1's multi_logloss: 0.339749
[200]	valid_0's multi_logloss: 0.38297	valid_1's multi_logloss: 0.298577
[300]	valid_0's multi_logloss: 0.383247	valid_1's multi_logloss: 0.267356
Early stopping, best iteration is:                    
[247]	valid_0's multi_logloss: 0.382912	valid_1's multi_logloss: 0.283153
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.385503	valid_1's multi_logloss: 0.339835
[200]	valid_0's multi_logloss: 0.381962	valid_1's multi_logloss: 0.298276
[300]	valid_0's multi_logloss: 0.382253	valid_1's multi_logloss: 0.266993
Early stopping, best iteration is:                    
[249]	valid_0's multi_logloss: 0.381808	valid_1's multi_logloss: 0.282088
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.382665	valid_1's multi_logloss: 0.340781
[200]	valid_0's multi_logloss: 0.378551

Score - 0.8875597528273289, Std - 0.0014793520349300454, Eval Score - 0.8875597528273289
Score across folds - [0.8853134351560374, 0.8879561618281451, 0.8896661614395087, 0.8866347984920913, 0.888228207220862].


...

Score - 0.889596206909953, Std - 0.0010380008568623045, Eval Score - 0.889596206909953
Score across folds - [0.888461388986048, 0.8898993432046948, 0.8908709338929696, 0.888305934475924, 0.8904434339901286].


  4%|▍         | 2/50 [04:50<1:59:07, 148.90s/trial, best loss: 0.8875597528273289]


Iteration: 3, Training with params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.9, 'learning_rate': 0.1, 'min_child_weight': 80.0, 'n_estimators': 5000, 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 0.6000000000000001, 'subsample_freq': 5}


Training until validation scores don't improve for 100 rounds                      
[100]	valid_0's multi_logloss: 0.384207	valid_1's multi_logloss: 0.327608          
[200]	valid_0's multi_logloss: 0.382115	valid_1's multi_logloss: 0.280778          
[300]	valid_0's multi_logloss: 0.383421	valid_1's multi_logloss: 0.245457          
Early stopping, best iteration is:                                                 
[206]	valid_0's multi_logloss: 0.381985	valid_1's multi_logloss: 0.278331
Training until validation scores don't improve for 100 rounds                      
[100]	valid_0's multi_logloss: 0.382623	valid_1's multi_logloss: 0.327971          
[200]	valid_0's multi_logloss: 0.380609	valid_1's multi_logloss: 0.280399          
Early stopping, best iteration is:                                                 
[178]	valid_0's multi_logloss: 0.380237	valid_1's multi_logloss: 0.289305
Training until validation scores don't improve for 100 rounds                      
  4%|▍      

KeyboardInterrupt: 